<a href="https://colab.research.google.com/github/fatemeh-ict/Deep-learnin/blob/main/eye_tracking_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install cvzone

  Preparing metadata (setup.py) ... done
  Created wheel for cvzone: filename=cvzone-1.6.1-py3-none-any.whl size=26297 sha256=dc5bc3edc462f392c828ac2c496a2000322da656a0196d5baef2c90ef4216412
  Stored in directory: /root/.cache/pip/wheels/2c/9f/b3/92e945ac4a71bf727a92463f38155cc5a4fa49c5010b38ec4c
Successfully built cvzone


In [4]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 29.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyl

In [8]:
import cv2
import numpy as np
from cvzone.FaceDetectionModule import FaceDetector
from cvzone.FaceMeshModule import FaceMeshDetector

In [ ]:
from google.colab.patches import cv2_imshow

LEFT_EYE =[362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398]


detector = FaceDetector()
meshdetector = FaceMeshDetector(maxFaces=1)


cap = cv2.VideoCapture("/content/drive/MyDrive/WIN_20240416_16_55_22_Pro.mp4")
if (cap.isOpened()== False):
    print("Error opening video stream or file")


while(cap.isOpened()):
    ret, frame = cap.read()
    ret, frame2 = cap.read()
    if ret == True:
        face_img, bbox = detector.findFaces(frame)
        face_img, faces = meshdetector.findFaceMesh(frame)
        print(bbox)
        if bbox:
            center = bbox[0]["center"]
            if faces:
                left_eye_points = np.array([[faces[0][p][0],faces[0][p][1]] for p in LEFT_EYE])
                #cv2.fillPoly(face_img2, pts=[left_eye_points], color = 255)
                (ex,ey,ew,eh) = cv2.boundingRect(left_eye_points)
                #cv2.rectangle(face_img2, (ex,ey), (ex+ew,ey+eh), (255,255,255))
                eye_roi = frame2[ey:ey+eh, ex:ex+ew]
                eye_roi_gr = cv2.cvtColor(eye_roi, cv2.COLOR_BGR2GRAY)
                _, iris = cv2.threshold(eye_roi_gr, 40, 255, cv2.THRESH_BINARY_INV)
                contours, _ = cv2.findContours(iris, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)

                if contours:
                    (ix,iy,iw,ih) = cv2.boundingRect(contours[0])
                    ix_cntr, iy_centr = ix+int(iw/2) + ex, iy+int(ih/2)+ey
                    cv2.circle(frame2, (ix_cntr, iy_centr), 5, (0,0,255), -1)

                    ix_cntr_e, iy_centr_e = ix+int(iw/2), iy+int(ih/2)

                    offset = 10
                    if ix_cntr_e > int(ew/2)+offset:
                        text = "right"
                    elif ix_cntr_e < int(ew/2)-offset:
                        text = "left"
                    else:
                        text = "center"

                    cv2.putText(frame2, text, (100,100), cv2.FONT_HERSHEY_PLAIN, 3, (0,60,0), 2)


        cv2_imshow(frame2)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    else:
        break

cap.release()
cv2.destroyAllWindows()